<a href="https://colab.research.google.com/github/milenaESSCA/Paris-G1-bousselmi-dubost/blob/main/ProjectPython_DiMemmo_ElMajhad_Samah_Dubost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# PACKAGE IMPORT DATA 
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import yfinance as yf
yf.pdr_override()
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# EXPLANATION OF THE THE MEANING OF THIS SERIE OF CODES: 

# Print a explanatory sentence
print("We are analyzing the stocks below to give our Investors insights on which stock to choose from this list:")

# The list where company name is stored
company_names = []

# Get company name from ticker name
tickers = ['OR.PA', 'CA.PA', 'BN.PA', 'SAN.PA','ORA.PA','RI.PA','MC.PA','SW.PA','ENGI.PA','CAP.PA']
for ticker in tickers:
    stock_data = yf.Ticker(ticker).info
    company_name = stock_data['longName']
    company_names.append(company_name)

# Print the company names
print(company_names)

# Print a explanatory sentence
print("Our analysis below is based on the stock prices of those companies between 2017 to 2022")


In [ ]:
# IMPORT DATA

# Create def to provide table with data from each ticker over a 5 year period 
def get(tickers, startdate, enddate):
    def data(ticker):
        return pdr.get_data_yahoo(ticker, start=startdate, end=enddate)
    datas = map (data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))

# The data chosen named all_data
all_data = get(tickers, datetime(2017, 1, 1), datetime(2022, 1, 1))
print(all_data)

In [ ]:
# GRAPH OF THE STOCK PRICE OVER TIME 

# select the 'Adj Close' column from the DataFrame
adj_close = all_data['Adj Close']

# plot the adjusted close price for each ticker
fig, ax = plt.subplots(figsize=(12,8))
for ticker in tickers:
    ax.plot(adj_close.loc[ticker], label=ticker)
ax.set_xlabel('Date')
ax.set_ylabel('Adjusted Close Price')
ax.set_title('Stock Prices')
ax.legend()
plt.show()

In [ ]:
# FREQUENCY DISTRIBUTION ON CLOSING PRICES / STOCKS

#Isolate the `Adj Close` values and transform the DataFrame
daily_close_px = adj_close.reset_index().pivot('Date', 'Ticker', 'Adj Close')

#Calculate the daily percentage change for `daily_close_px`
daily_pct_change = daily_close_px.pct_change()

#Plot the distributions
daily_pct_change.hist(bins=50, sharex=False, figsize=(12,8))

#Show the resulting plot
plt.show()

In [ ]:
# ANNUAL RETURN & STANDARD DEVIATION / STOCK

# The annual return / stock
annual_return = (1 + daily_pct_change.mean()) ** 252 - 1

# The standard deviation / stock 
std_dev = daily_pct_change.std() * np.sqrt(252)

# Plot data / ticker
ax = table.plot(kind='bar', rot=0)

# Set chart title and axis labels
plt.title('Compare each company return and volatility')
plt.xlabel('Ticker')
plt.ylabel('Percentage')

# Show plot
plt.show()

In [ ]:
# ANNUAL GROWTH RATE PER STOCK WITH VISUAL REPRESENTATION:

# Annual growth rates dataframe
annual_growth_rates = pd.DataFrame()

for ticker in tickers:
    data = all_data.loc[ticker]['Adj Close']
    prices_at_end_of_year = data.resample('Y').last()
    annual_growth_rates[ticker] = prices_at_end_of_year.pct_change()

# Plot of Annual growth rates / ticker
annual_growth_rates.plot()
plt.title('Annual Growth Rates')
plt.xlabel('Year')
plt.ylabel('Growth Rate')
plt.show()

In [ ]:
# ANNUAL RISK MAP PER STOCK (MEAN OF RETURNS)

# Create the risk map for each ticker
fig, axs = plt.subplots(nrows=len(tickers), figsize=(8, 5 * len(tickers)))

for i, ticker in enumerate(tickers):
    sns.scatterplot(x=std_dev, y=annual_return, ax=axs[i])
    #sns.scatterplot(x=std_returns, y=mean_returns, ax=axs[i])
    axs[i].set_title(ticker)

# Set the plot title and axis labels
fig.suptitle('Risk Map')
plt.xlabel('Standard Deviation')
plt.ylabel('Mean Return')

# Adjust spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:
#VOLATILITY CALCULATION

#Define the minumum of periods to consider
min_periods = 75

#Calculate the volatility
vol = daily_pct_change.rolling(min_periods).std() * np.sqrt(min_periods)

#Plot the volatility
vol.plot(figsize=(10, 8))

#Show the plot
plt.show()

In [ ]:
# MAXIMUM DRAWDOWN

# Define a trailing 252 trading day window
window = 252

# Max Drawdown dataframe
max_drawdowns = pd.DataFrame()

for ticker in tickers:
    data = all_data.loc[ticker]['Adj Close']
    # Calculate the max drawdown in the past window days for each day
    rolling_max = data.rolling(window, min_periods=1).max()
    daily_drawdown = data/rolling_max - 1.0
    # Calculate the minimum (negative) daily drawdown
    max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()
    max_drawdowns[ticker] = max_daily_drawdown
    
# Plot the results
max_drawdowns.plot()
# Show the plot
plt.show()


In [ ]:
# CREATE A SCATTERPLOT FOR EACH STOCK WITH DAILY RETURNS: 

# Create a figure with multiple subplots
fig, axs = plt.subplots(nrows=len(tickers), figsize=(8, 5 * len(tickers)))

# Iterate through each ticker and create a scatter plot
for i, ticker in enumerate(tickers):
    df = all_data.loc[ticker]
    sns.scatterplot(x=df.index, y='Adj Close', data=df, ax=axs[i])
    axs[i].set_title(ticker)

# Adjust spacing between subplots
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:
# FINAL SUMMARY TABLE 
summary_table = pd.DataFrame()
for ticker in tickers:
    data = all_data.loc[ticker]['Adj Close']
    volatility = std_dev[ticker]
    returns = annual_return[ticker]
    drawdown = max_drawdowns[ticker].min()
    summary_table.at[ticker, 'Mean Stock Price'] = data.mean()
    summary_table.at[ticker, 'Volatility'] = volatility
    summary_table.at[ticker, 'Return'] = returns
    summary_table.at[ticker, 'Growth Rate'] = annual_growth_rates.loc[:, ticker].mean()
    summary_table.at[ticker, 'Max drawdown'] = drawdown

print(summary_table)

In [ ]:
# Choose the best stock
summary_table = summary_table.sort_values(by=['Return', 'Volatility', 'Max drawdown'], ascending=[False, True, True])
best_ticker = summary_table.index[0]
print(f"The best performing ticker is {best_ticker}")